# **Công cụ hỗ trợ crawl dữ liệu**

Cài đặt BeautifulSoup

In [5]:
# %pip install beautifulsoup4
# %pip install requests


  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/57/ec/80c8d48ac8b1741d5b963797b7c0c869335619e13d4744ca2f67fc11c6fc/charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c962

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import thư viện

In [34]:
from bs4 import BeautifulSoup
import requests
import time
import csv


Đối tượng thể hiện mỗi bài tuyển dụng

In [35]:
class Job:
    def __init__(self, title, link, company, salary, locations, level, yearOfExperience, contractType, sex, skills, size_company):
        self.title = title
        self.link = link
        self.company = company
        self.salary = salary
        self.locations = locations
        self.level = level
        self.yearOfExperience = yearOfExperience
        self.contractType = contractType
        self.sex = sex
        self.skills = skills
        self.size_company = size_company
        

Hàm ghi một đối tượng công việc vào 1 hàng trong file .CSV

In [36]:
# raw_data_path = '../data/raw_data.csv'
raw_data_path = '../data/job_data_update_41-63.csv'

In [37]:
def write_to_csv(job):
    with open(raw_data_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            # Write header row if file is empty
            if file.tell() == 0:
                writer.writerow(['Title', 'Link', 'Company', 'Size company', 'Salary', 'Locations', 'Level', 'Year of Experience', 'Contract Type', 'Sex', 'Skills'])
            writer.writerow([job.title, job.link, job.company, job.size_company, job.salary, job.locations, job.level, job.yearOfExperience, job.contractType, job.sex, job.skills])

Hàm lấy thông tin chi tiết của mỗi bài tuyển dụng

In [38]:
def get_jobs(item):
    print('********************* Start')
    try:
        title = item.find(class_='title').text.strip()
        print(title)
        link = item.find(class_='title').find('a')['href'].strip()
        print(link)
        company = item.find(class_='company').text.strip()
        print(company)
        salary = item.find(class_='title-salary').text.strip()
        print(salary)
        locations = item.find(class_='address').text.strip()
        print(locations)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        # Số lần thử lại tối đa
        max_retries = 5
        retry_delay = 6  # Thời gian chờ giữa các lần thử lại, đơn vị là giây

        retry_count = 0
        while retry_count < max_retries:
            response2 = requests.get(link, headers=headers)
            if response2.status_code == 429:  # Nếu nhận được mã trạng thái 429
                print("Too many requests. Retrying after waiting...")
                retry_count += 1
                time.sleep(retry_delay)  # Chờ trước khi thử lại
            else:
                # Xử lý kết quả ở đây nếu cần
                # Ví dụ: print(response.text)
                break
        else:
            print("Max retries exceeded. Request unsuccessful.")
        print(response2.status_code)
        if response2.status_code == 200:
            print('********************* Success')
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if link.startswith("https://www.topcv.vn/brand/"):
                infos = soup2.find_all(class_='box-info')
                generals = infos[0].find_all('span')
                level = generals[3].text.strip()
                print(level)
                yearOfExperience = generals[5].text.strip()
                print(yearOfExperience)
                contractType = generals[2].text.strip()
                print(contractType)
                sex = generals[4].text.strip()
                print(sex)
                skills = infos[2].find(class_='content-tab').text.strip()
                print(skills)
            else:
                generals = soup2.find(class_='job-detail__box--right job-detail__body-right--item job-detail__body-right--box-general').find_all(class_='box-general-group-info-value')
                level = generals[0].text.strip()
                print(level)
                yearOfExperience = generals[1].text.strip()
                print(yearOfExperience)
                contractType = generals[3].text.strip()
                print(contractType)
                sex = generals[4].text.strip()
                print(sex)
                skills = soup2.find_all(class_='job-description__item--content')[1].text.strip()
                print(skills)
                size_company = soup2.find(class_='job-detail__company--information-item company-scale').find(class_='company-value').text.strip()
                print(size_company)
                
        else:
            level = None
            yearOfExperience = None
            contractType = None
            sex = None
            skills = None
        job = Job(title, link, company, salary, locations, level, yearOfExperience, contractType, sex, skills, size_company)
        # Writing to CSV file
        write_to_csv(job)
    except:
        print('********************* Fail')
    print('********************* End')

Hiện tại khuyến khích crawl mỗi lần 1 trang sau 1 khoảng thời gian rồi crawl tiếp trang khác để tránh lỗi 429

Crawl 1 trang

In [25]:
url = 'https://www.topcv.vn/viec-lam-it'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    jobs = soup.find_all(class_='job-item-2')
    for job in jobs:
        get_jobs(job)   
        time.sleep(5)
else:
    print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)

********************* Start
Nhân Viên SEO Youtube (Lương + Phụ Cấp + Thưởng Theo Doanh Thu Kênh Upto 100 Triệu)
https://www.topcv.vn/viec-lam/nhan-vien-seo-youtube-luong-phu-cap-thuong-theo-doanh-thu-kenh-upto-100-trieu/1321130.html?ta_source=ITJobs_LinkDetail
CÔNG TY TNHH CÔNG NGHỆ VÀ TRUYỀN THÔNG IGC VIỆT NAM
6 - 9 triệu
Hà Nội
200
********************* Success
Nhân viên
Không yêu cầu kinh nghiệm
Toàn thời gian
Không yêu cầu
- Nam / Nữ tốt nghiệp CĐ/ ĐH chuyên ngành liên quan: Marketing, Đồ họa,...
- Biết sử dụng cơ bản các phần mềm chỉnh sửa ảnh, edit video như Adobe Photoshop, Premiere, After Effect (Chấp nhận đào tạo thêm)
- Hiểu biết cơ bản về SEO từ khóa (Chấp nhận đào tạo thêm)
- Ưu tiên độ tuổi 1996 - 2001
- Yêu thích âm nhạc
- Có tư duy hình ảnh, bố cục và thẩm mỹ tốt, khả năng tự tìm tòi nghiên cứu.
- Làm việc nhanh nhẹn, chủ động trong công việc.
- Kiên trì, chăm chỉ, tự giác có trách nhiệm với công việc
- Có khả năng chịu áp lực cao, kỹ năng giải quyết vấn đề.
- Gửi các sả

Crawl toàn bộ trang

In [39]:
url = 'https://www.topcv.vn/viec-lam-it'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    num_pages = int(soup.find(class_='pagination').find_all('li')[-2].text)
    # num_pages = 10
    print(num_pages)
    # print('Num page'+ str(1))
    # jobs = soup.find_all(class_='job-item-2')
    # for job in jobs:
    #     get_jobs(job)
    #     time.sleep(5)
    for i in range(41, num_pages + 1):
        print('Num page'+ str(i))
        url = 'https://www.topcv.vn/viec-lam-it?page=' + str(i)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            jobs = soup.find_all(class_='job-item-2')
            for job in jobs:
                get_jobs(job)
                time.sleep(5)
        else:
            print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)   
        time.sleep(10)           
else:
    print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)

61
Num page41
********************* Start
Business Development Executive (Blockchain)
https://www.topcv.vn/viec-lam/business-development-executive-blockchain/1303349.html?ta_source=ITJobs_LinkDetail
ICETEA LABS
Tới 1,000 USD
Hà Nội
200
********************* Success
Nhân viên
2 năm
Toàn thời gian
Không yêu cầu
BA/BS degree in Marketing or related curriculumFrom 2 year of experience in Sales/Marketing while focusing on blockchain verticalDemonstrated understanding of the blockchain industry with strong existing industry network is a big plusExcellent verbal and written English and Local language communication skillsExperience with analyzing data and delivering insightsSelf-starter, fast learner with a can-do spiritHave foundational knowledge of Blockchain is a big plus
100-499 nhân viên
********************* End
********************* Start
IT Helpdesk
https://www.topcv.vn/viec-lam/it-helpdesk/1099734.html?ta_source=ITJobs_LinkDetail
CÔNG TY CỔ PHẦN TẬP ĐOÀN GOLDEN GATE – CHI NHÁNH MIỀN B

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
for i in range(35, 40 + 1):
        url = 'https://www.topcv.vn/viec-lam-it?page=' + str(i)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            jobs = soup.find_all(class_='job-item-2')
            for job in jobs:
                get_jobs(job)
                time.sleep(5)
        else:
            print('Yêu cầu không thành công. Mã trạng thái:', response.status_code)   
        time.sleep(10)    

********************* Start
Content Seo Website
https://www.topcv.vn/viec-lam/content-seo-website/1306214.html?ta_source=ITJobs_LinkDetail
CÔNG TY TNHH THỂ THAO THIÊN TRƯỜNG
8 - 11 triệu
Hà Nội
200
********************* Success
Nhân viên
1 năm
Toàn thời gian
Không yêu cầu
- Có kiến thức chuyên môn về SEO và Social Marketing (nếu chưa sẽ được đào tạo)- Có khả năng research, tổng hợp thông tin và viết content tốt- Biết chỉnh sửa ảnh, video cơ bản bằng PTS, Canva, Capcut,...là lợi thế (chưa biết sẽ được đào tạo)- Có khả năng làm việc nhóm và độc lập.
********************* End
********************* Start
Thực Tập Sinh Game Design
https://www.topcv.vn/viec-lam/thuc-tap-sinh-game-design/1305873.html?ta_source=ITJobs_LinkDetail
CÔNG TY CỔ PHẦN TECHASIANS
2 - 3 triệu
Hà Nội
200
********************* Success
Thực tập sinh
Không yêu cầu kinh nghiệm
Toàn thời gian
Nữ
Sinh viên năm cuối hoặc sinh viên mới ra trường các ngành Công nghệ thông tin, Phát triển game, Thiết kế game, Thiết kế sản phẩm và